In [1]:
import json
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sim.dy.model_intv import ModelIntv
import sim.dy.keys as I
import numpy.random as rd

In [2]:
n_collect = 200

In [3]:
np.linspace(0, 0.8, 33)

array([0.   , 0.025, 0.05 , 0.075, 0.1  , 0.125, 0.15 , 0.175, 0.2  ,
       0.225, 0.25 , 0.275, 0.3  , 0.325, 0.35 , 0.375, 0.4  , 0.425,
       0.45 , 0.475, 0.5  , 0.525, 0.55 , 0.575, 0.6  , 0.625, 0.65 ,
       0.675, 0.7  , 0.725, 0.75 , 0.775, 0.8  ])

In [23]:
def collect_vul(pars_post, n_collect=20, coverages=np.linspace(0, 0.8, 33), tp="VulACF"):
    idx = ['Pop', 'IncR', 'MorR', 
           'ACF_Vul_Footfall', 'ACF_Vul_Screened', 'ACF_Vul_Confirmed', 'ACF_Vul_Yield', 'ACF_Vul_TP',
           'ACF_Vul_DS_Fl', 'ACF_Vul_DR_Fl', 'ACF_Vul_DR_Sl', 
           'ACF_Plain_Footfall', 'ACF_Plain_Screened', 'ACF_Plain_Confirmed', 'ACF_Plain_Yield', 'ACF_Plain_TP', 
           'ACF_Plain_DS_Fl', 'ACF_Plain_DR_Fl', 'ACF_Plain_DR_Sl']
    
    m = ModelIntv()
    
    sel = rd.choice(list(range(len(pars_post))), n_collect)
    sel.sort()

    mss = list()
    mss_stats = list()


    for i in sel:
        pars = pars_post[i]
        y1, pars = m.find_baseline(pars, 2022)

        for k in coverages:    
            _, ms, _ = m.simulate_onward(y1, pars, intv={tp: {'Coverage': k}}, t_end=2031)

            ms = ms[idx].reset_index().assign(Key = i, Coverage=k, p_comorb=pars['p_comorb'], rr=pars['rr_risk_comorb'])
            mss.append(ms)
            
            ms_stats = {'Key': i, 'Coverage': k, 'Pop0': ms.Pop.iloc[0]}

            for key in idx[1:]:
                ms_stats[key] = (ms[key] * ms.Pop).sum()
            
            mss_stats.append(ms_stats)
            
    mss = pd.concat(mss).assign(Type=tp)
    mss_stats = pd.DataFrame(mss_stats).assign(Type=tp)
    return mss, mss_stats

In [24]:
for folder in ['dy_lo', 'dy_hi']:
    print(folder)
    out_path = f'out/{folder}'
    pars_post = json.load(open(f'{out_path}/Post.json', 'r'))
    r_acf = json.load(open(f'{out_path}/R_ACF_Bg.json', 'r'))

    for p in pars_post:
        p.update(r_acf)

    mss0, mss_stats0 = collect_vul(pars_post, n_collect=n_collect, tp='VulACF')
    mss1, mss_stats1 = collect_vul(pars_post, n_collect=n_collect, tp='PlainACF')
    mss = pd.concat([mss0, mss1])
    mss_stats = pd.concat([mss_stats0, mss_stats1])
    mss.to_csv(f'{out_path}/Sim_VulACF_budget.csv')
    mss_stats.to_csv(f'{out_path}/Sim_VulACF_budget_stats.csv')

dy_lo
dy_hi


In [25]:
def collect_fu(pars_post, n_collect=20, coverage=0.03, tp="VulACF"):
    idx = ['Pop', 'IncR', 'MorR', 
           'ACF_Vul_Footfall', 'ACF_Vul_Screened', 'ACF_Vul_Confirmed', 'ACF_Vul_Yield', 'ACF_Vul_TP',
           'ACF_Vul_DS_Fl', 'ACF_Vul_DR_Fl', 'ACF_Vul_DR_Sl', 
           'ACF_Plain_Footfall', 'ACF_Plain_Screened', 'ACF_Plain_Confirmed', 'ACF_Plain_Yield', 'ACF_Plain_TP', 
           'ACF_Plain_DS_Fl', 'ACF_Plain_DR_Fl', 'ACF_Plain_DR_Sl']
    
    intvs = {
        'Baseline': {},
        'Vul_0_0': {'VulACF': {'Coverage': coverage, 'FollowUp': 0, 'Duration': 0}},
        'Vul_2_3': {'VulACF': {'Coverage': coverage, 'FollowUp': 0.25, 'Duration': 2}},
        'Vul_2_6': {'VulACF': {'Coverage': coverage, 'FollowUp': 0.5, 'Duration': 2}},
        'Vul_3_3': {'VulACF': {'Coverage': coverage, 'FollowUp': 0.25, 'Duration': 3}},
        'Vul_3_6': {'VulACF': {'Coverage': coverage, 'FollowUp': 0.5, 'Duration': 3}},
    }
    
    
    m = ModelIntv()
    
    sel = rd.choice(list(range(len(pars_post))), n_collect)
    sel.sort()

    mss = list()
    mss_stats = list()


    for i in sel:
        pars = pars_post[i]
        y1, pars = m.find_baseline(pars, 2022)
        
        for key, intv in intvs.items():
            _, ms, _ = m.simulate_onward(y1, pars, intv=intv, t_end=2031)

            ms = ms[idx].reset_index().assign(Key = i, p_comorb=pars['p_comorb'], rr=pars['rr_risk_comorb'])
            mss.append(ms)
            
            ms_stats = {'Key': i, 'Scenario': key, 'Pop0': ms.Pop.iloc[0]}

            for key in idx[1:]:
                ms_stats[key] = (ms[key] * ms.Pop).sum()
            
            mss_stats.append(ms_stats)
            
    mss = pd.concat(mss).assign(Coverage = coverage)
    mss_stats = pd.DataFrame(mss_stats).assign(Coverage = coverage)
    return mss, mss_stats

In [26]:
for folder in ['dy_lo', 'dy_hi']:
    print(folder)
    out_path = f'out/{folder}'
    pars_post = json.load(open(f'{out_path}/Post.json', 'r'))
    r_acf = json.load(open(f'{out_path}/R_ACF_Bg.json', 'r'))

    for p in pars_post:
        p.update(r_acf)
        
    mss, mss_stats = collect_fu(pars_post, n_collect=n_collect, coverage=0.03)
    mss.to_csv(f'{out_path}/Sim_VulACF_followup.csv')
    mss_stats.to_csv(f'{out_path}/Sim_VulACF_followup_stats.csv')

dy_lo
dy_hi


In [27]:
mss_stats

,Key,Scenario,Pop0,IncR,MorR,ACF_Vul_Footfall,ACF_Vul_Screened,ACF_Vul_Confirmed,ACF_Vul_Yield,ACF_Vul_TP,...,ACF_Vul_DR_Sl,ACF_Plain_Footfall,ACF_Plain_Screened,ACF_Plain_Confirmed,ACF_Plain_Yield,ACF_Plain_TP,ACF_Plain_DS_Fl,ACF_Plain_DR_Fl,ACF_Plain_DR_Sl,Coverage
0,0,Baseline,2.383050,0.194895,0.023667,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.03
1,0,Vul_0_0,2.383050,0.187833,0.022209,7.432256,1.322301,0.352132,0.018782,0.008566,...,0.000350,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.03
2,0,Vul_2_3,2.383050,0.177978,0.020064,9.444957,1.322301,0.351963,0.018574,0.008357,...,0.000346,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.03
3,0,Vul_2_6,2.383050,0.180209,0.020557,9.444754,1.322301,0.352117,0.018763,0.008547,...,0.000353,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.03
4,0,Vul_3_3,2.383050,0.175795,0.019546,10.417183,1.322301,0.351935,0.018539,0.008321,...,0.000347,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.03
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1195,1497,Vul_0_0,2.382899,0.153185,0.018408,7.426154,1.322217,0.311921,0.020970,0.006105,...,0.000263,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.03
1196,1497,Vul_2_3,2.382899,0.146635,0.016970,9.422772,1.322217,0.311869,0.020908,0.006042,...,0.000262,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.03
1197,1497,Vul_2_6,2.382899,0.147999,0.017292,9.422606,1.322217,0.311969,0.021027,0.006162,...,0.000267,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.03
1198,1497,Vul_3_3,2.382899,0.145117,0.016609,10.383915,1.322217,0.311856,0.020893,0.006027,...,0.000263,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.03


In [8]:
def collect_fux(pars_post, n_collect=20, coverage=0.03, tp="VulACF"):
    idx = ['Pop', 'IncR', 'MorR', 
           'ACF_Vul_Footfall', 'ACF_Vul_Screened', 'ACF_Vul_Confirmed', 'ACF_Vul_Yield', 'ACF_Vul_TP',
           'ACF_Vul_DS_Fl', 'ACF_Vul_DR_Fl', 'ACF_Vul_DR_Sl', 
           'ACF_Plain_Footfall', 'ACF_Plain_Screened', 'ACF_Plain_Confirmed', 'ACF_Plain_Yield', 'ACF_Plain_TP', 
           'ACF_Plain_DS_Fl', 'ACF_Plain_DR_Fl', 'ACF_Plain_DR_Sl']
    
    
    m = ModelIntv()
    
    sel = rd.choice(list(range(len(pars_post))), n_collect)
    sel.sort()

    mss = list()
    mss_stats = list()


    for i in sel:
        pars = pars_post[i]
        y1, pars = m.find_baseline(pars, 2022)
    
        _, ms, _ = m.simulate_onward(y1, pars, intv={}, t_end=2031)

        ms = ms[idx].reset_index().assign(Key=i, FollowUp=0, Duration=0)
        mss.append(ms)
        ms_stats = {'Key': i, 'Scenario': 'Baseline', 'Pop0': ms.Pop.iloc[0], 
                    'FollowUp': 0, 'Duration': 0}
        for key in idx[1:]:
            ms_stats[key] = (ms[key] * ms.Pop).sum()

        mss_stats.append(ms_stats)
    
        for fu in np.linspace(0, 1, 13):
            for dur in np.linspace(0, 4, 9):
                intv = {'VulACF': {'Coverage': coverage, 'FollowUp': fu, 'Duration': dur}}
                
                _, ms, _ = m.simulate_onward(y1, pars, intv=intv, t_end=2031)

                ms = ms[idx].reset_index().assign(Key = i, FollowUp=fu, Duration=dur)
                mss.append(ms)

                ms_stats = {'Key': i, 'Scenario': key, 'Pop0': ms.Pop.iloc[0], 
                            'FollowUp': fu, 'Duration': dur}

                for key in idx[1:]:
                    ms_stats[key] = (ms[key] * ms.Pop).sum()

                mss_stats.append(ms_stats)
            
    mss = pd.concat(mss).assign(Coverage = coverage)
    mss_stats = pd.DataFrame(mss_stats).assign(Coverage = coverage)
    return mss, mss_stats

In [9]:
for folder in ['dy_hi']:
    print(folder)
    out_path = f'out/{folder}'
    pars_post = json.load(open(f'{out_path}/Post.json', 'r'))
    r_acf = json.load(open(f'{out_path}/R_ACF_Bg.json', 'r'))

    for p in pars_post:
        p.update(r_acf)
        
    mss, mss_stats = collect_fux(pars_post, n_collect=20, coverage=0.03)
    mss.to_csv(f'{out_path}/Sim_VulACF_fudur.csv')
    mss_stats.to_csv(f'{out_path}/Sim_VulACF_fudur_stats.csv')

dy_hi


In [7]:
mss_stats

,Key,Scenario,Pop0,FollowUp,Duration,IncR,MorR,ACF_Vul_Footfall,ACF_Vul_Screened,ACF_Vul_Confirmed,...,ACF_Vul_DR_Sl,ACF_Plain_Footfall,ACF_Plain_Screened,ACF_Plain_Confirmed,ACF_Plain_Yield,ACF_Plain_TP,ACF_Plain_DS_Fl,ACF_Plain_DR_Fl,ACF_Plain_DR_Sl,Coverage
0,115,Baseline,2.383219,0.0,0.0,0.169381,0.015161,0.000000,0.000000,0.000000,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.03
1,115,ACF_Plain_DR_Sl,2.383219,0.0,0.0,0.163501,0.014296,7.433538,1.322395,0.323625,...,0.000270,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.03
2,115,ACF_Plain_DR_Sl,2.383219,0.0,0.5,0.163501,0.014296,7.433538,1.322395,0.323625,...,0.000270,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.03
3,115,ACF_Plain_DR_Sl,2.383219,0.0,1.0,0.163501,0.014296,7.433538,1.322395,0.323625,...,0.000270,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.03
4,115,ACF_Plain_DR_Sl,2.383219,0.0,1.5,0.163501,0.014296,7.433538,1.322395,0.323625,...,0.000270,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.03
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1275,1459,ACF_Plain_DR_Sl,2.383424,1.0,2.0,0.160204,0.021759,9.435873,1.322508,0.297062,...,0.000396,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.03
1276,1459,ACF_Plain_DR_Sl,2.383424,1.0,2.5,0.159564,0.021584,9.928874,1.322508,0.297064,...,0.000397,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.03
1277,1459,ACF_Plain_DR_Sl,2.383424,1.0,3.0,0.159050,0.021440,10.412991,1.322508,0.297067,...,0.000398,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.03
1278,1459,ACF_Plain_DR_Sl,2.383424,1.0,3.5,0.158784,0.021354,10.857341,1.322508,0.297080,...,0.000400,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.03
